In [1]:
from modules.crawler import download

import json
import re
import sqlite3

In [2]:
conn = sqlite3.connect('hub.db')
cur = conn.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS medicen;
    CREATE TABLE  medicen
    (
        id INTEGER NOT NULL PRIMARY KEY,
        code TEXT NOT NULL,
        category_id INTEGER NULL,
        category_name TEXT NULL,
        name TEXT NOT NULL,
        effect_list TEXT NULL,
        summary TEXT NULL
    );
''')

In [3]:
url = 'https://herbhub.com/api/search/herb'
sql = '''
INSERT INTO medicen (
            id, code, category_id, category_name, name, effect_list, summary)
SELECT ?, ?, ?, ?, ?, ?, ?
WHERE NOT EXISTS (
    SELECT *
    FROM medicen
    WHERE id = ?
);
'''

In [4]:
with open('query.txt') as fp:
    queryList = fp.readlines()

searchResult = [
    download(url, params={
        'q': query,
        't': 'herb'
    }) for query in queryList
]
for result in searchResult:
    for item in result.json()['herb']:
        cur.execute(sql,
                        (item['id'], item['code'], item['category_id'],
                         item['category_name'], item['name_full'],
                         item['effect_list'], item['summary'], item['id']))
        conn.commit()

In [5]:
cur.execute('''
    SELECT * FROM medicen
    ORDER BY id ASC;
''')
cur.fetchall()

[(75, 'M0005389', None, None, '안지(雁脂)', '익기해독,서근활혈', None),
 (283, 'M0004914', None, None, '백모등(白毛藤)', '거풍해독,청열이습', None),
 (329,
  'M0001313',
  32,
  '거풍습지비통약(祛風濕止痺痛藥)',
  '방기(防己)A',
  '거풍지통',
  '습사(濕邪)가 편성하여 지체가 중통마목(重痛痲木)하고 관절이 부종한 착비(着痹)증상에 사용하는 약물이다.'),
 (330,
  'M0001314',
  32,
  '거풍습지비통약(祛風濕止痺痛藥)',
  '방기(防己)B',
  '거풍지통',
  '습사(濕邪)가 편성하여 지체가 중통마목(重痛痲木)하고 관절이 부종한 착비(着痹)증상에 사용하는 약물이다.'),
 (331,
  'M0001315',
  32,
  '거풍습지비통약(祛風濕止痺痛藥)',
  '방기(防己)C',
  '거풍지통',
  '습사(濕邪)가 편성하여 지체가 중통마목(重痛痲木)하고 관절이 부종한 착비(着痹)증상에 사용하는 약물이다.'),
 (332,
  'M0001316',
  32,
  '거풍습지비통약(祛風濕止痺痛藥)',
  '방기(防己)D',
  '거풍지통',
  '습사(濕邪)가 편성하여 지체가 중통마목(重痛痲木)하고 관절이 부종한 착비(着痹)증상에 사용하는 약물이다.'),
 (658, 'M0003626', None, None, '천금등(千金藤)B', '거풍,거풍,소종,이습,해열', None),
 (1810,
  'M0000200',
  18,
  '활혈거어약(活血祛瘀藥)',
  '계혈등(鷄血藤)',
  '활혈서근,양혈조경',
  '부인과 질환에 허실을 막론하고 응용하며, 서근활락효능이 있어 지체마목(肢體麻木), 근맥구련(筋脈拘攣) 등도 치료한다.'),
 (1974, 'M0004166', None, None, '해풍등(海風藤)', '거풍습,지비통,통경락', None),
 (3348,
  'M0001193',
  None,
  None,
  '목방기(木防

In [6]:
cur.close()
conn.close()